In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image

#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_standings.standings_05_onward`
# WHERE SEASON='2023-24'
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')
df_bq2=df_bq2.sort_values(by=["TeamName"])

Downloading: 100%|██████████|


In [2]:
df_bq2.head()

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS,Season
435,00,22012,1610612755,Philadelphia,76ers,East,22-30,9,- o,Atlantic,...,None,None,None,None,1-0,9-6,4-11,22-29,12-19,2012-13
574,00,22006,1610612755,Philadelphia,76ers,East,24-28,10,- x,Atlantic,...,None,None,None,None,None,5-9,3-13,17-36,18-11,2006-07
344,00,22020,1610612755,Philadelphia,76ers,East,31-11,1,- e,Atlantic,...,None,None,None,None,None,None,4-1,24-12,25-11,2020-21
21,00,22005,1610612755,Philadelphia,76ers,East,22-30,9,- o,Atlantic,...,None,None,None,None,None,8-8,7-7,25-27,13-17,2005-06
345,00,22020,1610612755,Philadelphia,76ers,East,31-11,1,- e,Atlantic,...,None,None,None,None,None,None,4-1,24-12,25-11,2020-21


In [ ]:
year='2023-24'

In [4]:
df_bq=df_bq2[(df_bq2["Season"]=='2015-16')&(df_bq2["Season"]==year)]

In [6]:
df_bq=df_bq.sort_values(by=["WINS"], ascending=False)

In [9]:
fig = px.bar(
  df_bq, 
  x="TeamName", 
  y="WINS",
  title="<b>Team Standings for the {} Season</b>".format(year),
  template="plotly_white",
  hover_data=['Conference','Record','ConferenceRecord','DivisionRecord', 'WINS', 'LOSSES', 'WinPCT', 'HOME', 'ROAD', 'L10', 'OT', 
              'ThreePTSOrLess', 'TenPTSOrMore', 'LongHomeStreak','LongRoadStreak','LongWinStreak', 'LongLossStreak','AheadAtHalf', 'BehindAtHalf', 'TiedAtHalf', 'AheadAtThird',
              'BehindAtThird', 'TiedAtThird', 'Score100PTS', 'OppScore100PTS','OppOver500', 'LeadInFGPCT', 'LeadInReb', 'FewerTurnovers', 'PreAS', 'PostAS', 'Season'], 
  height=700, 
  width=1200,
  color="TeamName",
  color_discrete_map={
"Hawks":"#C8102E",
"Bobcats":"#E35205",
"Nets":"#707372",
"Celtics":"#007A33",
"SuperSonics":"#00653A",
"Hornets":"#00778B",
"Braves":"#418FDE",
"Bulls":"#BA0C2F",
"Cavaliers":"#6F263D",
"Mavericks":"#0050B5",
"Nuggets":"#0C2340",
"Pistons":"#1D4289",
"Warriors":"#1D4289",
"Rockets":"#BA0C2F",
"Pacers":"#041E42",
"Clippers":"#0C2340",
"Lakers":"#6a00ed",
"Grizzlies":"#0C2340",
"Heat":"#862633",
"Bucks":"#2C5234",
"Timberwolves":"#0C2340",
"Pelicans":"#0C2340",
"Knicks":"#FF8200",
"Thunder":"#0072CE",
"Magic":"#0057B7",
"76ers":"#1D4289",
"Suns":"#211747",
"Trail Blazers":"#C8102E",
"Kings":"#965ddd",
"Spurs":"#9EA2A2",
"Raptors":"#C8102E",
"Jazz":"#0C2340",
"Wizards":"#C8102E",
"Royals":"#003DA5",
"Bullets":"#C8102E"
          },
  )

for x,y in zip(df_bq.TeamName, df_bq.WINS):
  for png in (Path.cwd().joinpath("NBA_LOGOS").glob("*.png")):
    if str.split(str.split(str(png),".")[0],"/")[6]==x:
      image=str(png)
      fig.add_layout_image(
          x=x,
          y=y+5,
          source=Image.open(image),
          xref="x",
          yref="y",
          sizex=6,
          sizey=6,
          xanchor="center",
          yanchor="middle",
      )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis_range=[0,85],
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_layout(showlegend=False)
fig

In [ ]:
fig.write_html("NBA Standings For The {} Season".format(year),full_html=False, include_plotlyjs='cdn')